# Extra Credit

For upto an additional 15 extra credit points on the midterm exam, you can write another function that applies additional criteria on top of what the midterm already did.  To earn the addition points, you need write a function called `summarize_results(patients)` to take the output of your  `process_people()` function to apply a new business rule and summarize the results.

* Your function should take the dictionary created by `process_people()` as it's one input parameter: `patients`.
* If the PRIEST probably is less than 10%, change the `hospital` and `address` to both have a value of `'Home'` indicating patients with a lower probability of an adverse outcome can just go home rather than be admitted to a hospital.
* Summarize those results and count how many patients are now being sent to each location.  Your result should have the following format:
```
{
    "Home": 123,
    "Hospital A": 10,
    "Hospital B": 20,
    "Hospital C": 30
}
```

As always, your function needs good docstrings; but does not require any doctests in this case.

In [13]:
import pandas as pd
import requests

In [14]:
def priest(patients_sex, patients_age, patients_resp, patients_os, patients_heart, patients_bp, patients_temp, patients_alert, patients_inspo, patients_status):
    """(str, int, int, int, int, float, float, str, str, str) -> (float)
    Compute a risk percentage using the algorithm from MDCalc portal.

    >>> priest('Male', 81, 27, 0.91, 40, 111, 36.1, 'Alert', 'AIR', 'Unrestricted normal activity')
    0.47

    >>> priest('MALE', 66, 12, 0.97, 132, 80, 34.5, 'Alert', 'Supplemental oxygen', 'Bed/chair bound, no self-care')
    0.59

    >>> priest('MALE', 16, 8, 0.89, 51, 169, 38.9, 'Confused or not alert', 'Supplemental oxygen', 'Unrestricted normal activity')
    0.46
    """
    risk_count = 0
    last_risk = 0

    # 'casefold()' method is stronger(https://www.w3schools.com/python/ref_string_casefold.asp)
    patients_status = patients_status.casefold()
    patients_sex = patients_sex.casefold()
    patients_alert = patients_alert.casefold()
    patients_inspo = patients_inspo.casefold()

    # check
    if patients_status not in ["unrestricted normal activity", "limited strenuous activity, can do light activity",
                               "limited activity, can self-care", "limited self-care", "bed/chair bound, no self-care"]:
        return None
    if patients_sex not in ["male", "female"]:
        return None
    if patients_alert not in ["alert", "confused or not alert"]:
        return None
    if patients_inspo not in ["air", "supplemental oxygen"]:
        return None

    # Sex
    if patients_sex == "male":
        risk_count += 1

    # Age
    if patients_age > 80:
        risk_count += 4
    elif patients_age >= 66:
        risk_count += 3
    elif patients_age >= 50:
        risk_count += 2

    # Respiratory rate
    if patients_resp < 9:
        risk_count += 3
    if patients_resp > 24:
        risk_count += 3
    elif 21 <= patients_resp <= 24:
        risk_count += 2
    elif 9 <= patients_resp <= 11:
        risk_count += 1

    # Oxygen saturation
    if patients_os < 0.92:
        risk_count += 3
    elif patients_os <= 0.93:
        risk_count += 2
    elif patients_os <= 0.95:
        risk_count += 1

    # Heart rate
    if patients_heart > 130:
        risk_count += 3
    elif patients_heart < 41:
        risk_count += 3
    elif 50 >= patients_heart >= 41 or 110 >= patients_heart >= 91:
        risk_count += 1
    elif 130 >= patients_heart >= 111:
        risk_count += 2

    # Systolic BP
    if patients_bp < 91 or patients_bp > 219:
        risk_count += 3
    elif 91 <= patients_bp <= 100:
        risk_count += 2
    elif 101 <= patients_bp <= 110:
        risk_count += 1

    # Temperature
    if patients_temp < 35.1:
        risk_count += 3
    elif 35.1 <= patients_temp <= 36.0:
        risk_count += 1
    elif 38.1 <= patients_temp <= 39.0:
        risk_count += 1
    elif 36.1 <= patients_temp <= 38.0:
        risk_count += 0
    else:
        risk_count += 2

    # Alertness
    if patients_alert == "confused or not alert":
        risk_count += 3

    # Inspired oxygen
    if patients_inspo == "supplemental oxygen":
        risk_count += 2

    # Performance status
    if patients_status == "bed/chair bound, no self-care":
        risk_count += 4
    elif patients_status == "limited strenuous activity, can do light activity":
        risk_count += 1
    elif patients_status == "limited activity, can self-care":
        risk_count += 2
    elif patients_status == "limited self-care":
        risk_count += 3

    # Risk percentage
    risk_map = {0: 0.01, 1: 0.01, 2: 0.02, 3: 0.02, 4: 0.03, 5: 0.09, 6: 0.15, 7: 0.18, 8: 0.22, 9: 0.26, 10: 0.29,
                    11: 0.34, 12: 0.38, 13: 0.46, 14: 0.47, 15: 0.49, 16: 0.55}

    if risk_count in risk_map:
        last_risk += risk_map[risk_count]
    elif 17 <= risk_count <= 25:
        last_risk += 0.59
    else:
        last_risk += 0.99

    # Return
    return last_risk


In [15]:
import requests

def find_hospital(age, sex, priest):
  """(int, str, float) -> (str)
  The function retrieves the name of the hospital by accessing a web service.

  >>> find_hospital(30, 'male', 0.10)
  'Southwest Hospital and Medical Center'

  >>> find_hospital(30, 'female', 0.20)
  'Select Specialty Hospital - Northeast Atlanta'

  >>> find_hospital(68, 'male', 0.30)
  'Wesley Woods Geriatric Hospital'
  """
  # REST web service
  api = 'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={}&sex={}&risk_pct={}'.format(age, sex, priest)
  request = requests.get(api)

  hospital_json = request.json()
  hospital = hospital_json.get('hospital')

  # Check
  if hospital_json:
    return hospital
  else:
    return None

In [16]:
import requests
import json

def get_address(hospital):
  """(str) -> (str)
   Function will acquire the hospital's address from json

   >>> get_address('PHOEBE SUMTER MEDICAL CENTER')
   '126 HIGHWAY 280 W'

   >>> get_address('ATHENS REGIONAL MED CENTER')
   '1199 PRINCE AVENUE'

   >>> get_address('ST MARYS HEALTH CARE SYSTEM')
   '1230 BAXTER STREET'
  """

  # Load
  with open('ga_hospitals.json') as ga:
    hospital_json = json.load(ga)

  address = hospital_json[hospital]['ADDRESS']

  # Check
  if hospital in hospital_json:
    return address
  else:
    return None

In [17]:
import pandas as pd

def process_people(path):
  """(path) -> (dict)
   Function wil use path provided and returns a list of dictionaries.
  """
  last_return = {}

  # Load
  csv = pd.read_csv(path, sep='|')

  # Loop using 'iterrows()'(https://www.w3schools.com/python/pandas/ref_df_iterrows.asp)
  for index, row in csv.iterrows():
    patients_code = row['patient']
    patients_sex = row['sex']
    patients_age = row['age']
    patients_resp = row['breath']
    patients_os = row['o2 sat']
    patients_heart = row['heart']
    patients_bp = row['systolic bp']
    patients_temp = row['temp']
    patients_alert = row['alertness']
    patients_inspo = row['inspired']
    patients_status = row['status']

    calculate_risk = priest(patients_sex, patients_age, patients_resp, patients_os, patients_heart, patients_bp, patients_temp, patients_alert, patients_inspo, patients_status)
    hospital_name = find_hospital(patients_age, patients_sex, calculate_risk)
    hospital_address = get_address(hospital_name.upper())

    last_return[patients_code] = [
        patients_sex, patients_age, patients_resp, patients_os, patients_heart, patients_bp, patients_temp, patients_alert, patients_inspo, patients_status, calculate_risk, hospital_name, hospital_address
    ]

  return last_return

In [18]:
def summarize_results(final):
    """(dict) -> (dict)
    updates the 'hospital' and 'address' to 'Home' for risk under 10% and summarizes at the end .
    """
    sumz = {}
    for i in final.values():
      if i[11] not in sumz:
        sumz[i[11]] = 0
      if i[10] < 0.1:
         i[11] = "Home"
         i[12] = "Home"
      else:
         sumz[i[11]] += 1

      sumz["Home"] = sum(1 for i in final.values() if i[11] == "Home")

    return sumz

In [19]:
summ = process_people('people.psv')

In [20]:
summarize_results(summ)

{'Select Specialty Hospital - Northeast Atlanta': 32,
 'Home': 24,
 'Wesley Woods Geriatric Hospital': 20,
 'Southwest Hospital and Medical Center': 16,
 'Childrens Healthcare of Atlanta at Scottish Rite': 1,
 'Hughes Spalding Childrens Hospital': 1,
 'Emory Dunwoody Medical Center': 5}